<a href="https://colab.research.google.com/github/pinkdolphin11/ESAA/blob/main/HW_0923.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 7. 앙상블 학습과 랜덤 포레스트

일련의 예측기(분류나 회귀 모델)로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있다. 일련의 예측기를 앙상블이라고 하고, 이를 앙상블 학습이라고 한다.

랜덤 포레스트 : 결정 트리의 앙상블. 훈련 세트로부터 무작위로 각기 다른 서브셋을 만들어 일련의 결정 트리 분류기를 훈련시킨다. 모든 개별 트리의 예측을 구한 다음 가장 많은 선택을 받은 클래스를 예측으로 삼는다.

## 7.1 투표 기반 분류기

직접 투표 분류기 : 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측. 다수결 투표로 정해지는 분류기.

다수결 투표 분류기가 앙상블에 포함된 개별 분류기 중 가장 뛰어난 것보다 높은 정확도를 보일 때가 많다. 각 분류기가 약한 학습기일지라도, 충분히 많고 다양하다면 앙상블은 강한 학습기가 될 수 있다. 이는 큰 수의 법칙 원리로 이해할 수 있다. 하지만 이런 가정은 모든 분류기가 완벽하게 독립적이고 오차에 상관관계가 없어야 가능하다. 하지만 같은 데이터로 훈련시킬 경우 이 가정이 맞지 않는다. 분류기들이 같은 종류의 오차를 만들기 쉽기 때문에 잘못된 클래스가 다수인 경우가 많아지고 앙상블의 정확도가 낮아진다. 앙상블 방법은 예측기가 가능한 한 독립적일 때 최고의 성능을 발휘하므로, 다양한 분류기를 얻기 위해 각기 다른 알고리즘으로 학습시켜야 한다.

In [1]:
# moons 데이터 불러오기

from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)

# train, test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [2]:
# 여러 분류기를 조합하여 사이킷런의 투표 기반 분류기(VotingClassifier)를 만들고 훈련시킴

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

voting_clf = VotingClassifier(estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)],voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [3]:
# 각 분류기의 테스트셋 정확도

from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
  clf.fit(X_train,y_train)
  y_pred = clf.predict(X_test)
  print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.888
SVC 0.896
VotingClassifier 0.896


투표 기반 분류기가 다른 개별 분류기보다 성능이 높다.

간접 투표 : 모든 분류기가 클래스의 확률을 예측할 수 있으면(predict_proba() 메서드가 있으면) 개별 분류기 예측들의 평균을 구해서 확률이 가장 높은 클래스를 예측할 수 있다. 확률이 높은 투표에 비중을 더 두기 때문에 직접 투표 방식보다 성능이 높다. voting='soft'로 지정해서 사용한다. SVC의 경우 probability=True로 추가적으로 지정해야 한다.

## 7.2 배깅과 페이스팅

다양한 분류기를 만드는 방법으로 각기 다른 훈련 알고리즘을 사용하는 것 말고도, 같은 알고리즘을 사용하고 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습시키는 것이 있다.

배깅 : 훈련 세트에서 중복을 허용하여 샘플링하는 방식

페이스팅 : 훈련 세트에서 중복을 허용하지 않고 샘플링하는 방식

배깅, 페이스팅에서는 같은 훈련 샘플을 여러 개의 예측기에 걸쳐 사용할 수 있지만 배깅만이 한 예측기를 위해 같은 훈련 샘플을 여러 번 샘플링할 수 있다.

모든 예측기가 훈련을 마치면 앙상블은 모든 예측을 모아서 새로운 샘플에 대한 예측을 만드는데, 분류의 경우 통계적 최빈값, 회귀에 대해서는 평균을 계산한다. 이런 수집 함수를 통과하면 개별 예측기의 편향과 분산이 모두 감소한다. 일반적으로 앙상블의 결과는 원본 데이터셋으로 하나의 예측기를 훈련시킬 때와 비교해 편향은 비슷하지만 분산은 주러든다.

### 7.2.1 사이킷런의 배깅과 페이스팅

In [4]:
# 결정 트리 분류기 500개의 앙상블을 훈련시킴

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(),n_estimators=500,max_samples=100,bootstrap=True,n_jobs=1)
bag_clf.fit(X_train,y_train)
y_pred = bag_clf.predict(X_test)

### 7.2.2 oob 평가

oob(out of bag) 샘플 : 배깅에서 선택되지 않은 나머지 훈련 샘플. 예측기마다 모두 다르다.

예측기가 훈련되는 동안은 oob 샘플을 사용하지 않으므로 별도의 검증 세트를 사용하지 않고 oob 샘플을 사용해 평가할 수 있다. 앙상블의 평가는 각 예측기의 oob 평가를 평균해서 얻는다.

In [5]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(),n_estimators=500,bootstrap=True,n_jobs=-1,oob_score=True) #oob_score=True : 훈련이 끝난 후 자동으로 oob 평가 수행
bag_clf.fit(X_train,y_train)
bag_clf.oob_score_ #평가 점수 결과는 oob_score_에 저장됨

0.9013333333333333

In [6]:
from sklearn.metrics import accuracy_score
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.896

위와 비슷한 정확도를 얻었다.

In [8]:
bag_clf.oob_decision_function_ #각 훈련 샘플의 클래스 확률 반환

array([[0.36413043, 0.63586957],
       [0.31791908, 0.68208092],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.07894737, 0.92105263],
       [0.38095238, 0.61904762],
       [0.02162162, 0.97837838],
       [0.98930481, 0.01069519],
       [0.96907216, 0.03092784],
       [0.76966292, 0.23033708],
       [0.00512821, 0.99487179],
       [0.75274725, 0.24725275],
       [0.81920904, 0.18079096],
       [0.96756757, 0.03243243],
       [0.0273224 , 0.9726776 ],
       [0.        , 1.        ],
       [0.96629213, 0.03370787],
       [0.95      , 0.05      ],
       [0.98876404, 0.01123596],
       [0.01010101, 0.98989899],
       [0.40223464, 0.59776536],
       [0.88571429, 0.11428571],
       [1.        , 0.        ],
       [0.98314607, 0.01685393],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.60233918, 0.39766082],
       [0.

## 7.3 랜덤 패치와 랜덤 서브스페이스

랜덤 패치 방식 : 훈련 특성과 샘플을 모두 샘플링하는 것

랜덤 서브스페이스 방식 : 훈련 샘플을 모두 사용하고(bootstrap=False, max_samples=1.0) 특성은 샘플링(bootstrap_feaures=True 또는 max_features<1.0)하는 것

특성 샘플링은 더 다양한 예측기를 만들며 편향을 늘리는 대신 분산을 낮춘다.

## 7.4 랜덤 포레스트

In [9]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500,max_leaf_nodes=16,n_jobs=-1)
rnd_clf.fit(X_train,y_train)
y_pred_clf = rnd_clf.predict(X_test)

In [11]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(max_features='auto',max_leaf_nodes=16),n_estimators=500,max_samples=1.0,bootstrap=True,n_jobs=-1)

### 7.4.1 엑스트라 트리

 

익스트림 랜덤 트리 앙상블(엑스트라 트리) : 극단적으로 무작위한 트리의 랜덤 포레스트. 편향이 늘어나지만 분산은 낮아진다.

### 7.4.2 특성 중요도

In [13]:
from sklearn.datasets import load_iris
iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500,n_jobs=-1)
rnd_clf.fit(iris['data'],iris['target'])
for name, score in zip(iris['feature_names'],rnd_clf.feature_importances_):
  print(name,score)

sepal length (cm) 0.08823272404432969
sepal width (cm) 0.021101362932409475
petal length (cm) 0.45626423147092715
petal width (cm) 0.4344016815523336


## 7.5 부스팅

부스팅 : 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법. 앞의 모델을 보완해나가면서 일련의 예측기를 학습시키는 것이다.

## 7.5.1 에이다부스트

이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높이면 새로운 예측기는 학습하기 어려운 샘플에 점점 맞춰지게 되는데, 이를 에이다부스트라고 한다. 예를 들어, 에이다부스트 분류기를 만들 때, 먼저 알고리즘이 기반이 되는 첫번째 분류기를 훈련 세트에서 훈련시키고 예측을 만든다. 그 다음 알고리즘이 잘못 분류된 훈련 샘플의 가중치를 상대적으로 높인다. 두 번째 분류기는 업데이트된 가중치를 사용해 훈련 세트에서 훈련하고 다시 예측을 만든다. 이런 식으로 가중치 업데이트를 반복한다.

<예측기 가중치>

$\alpha_{j} = 학습률 하이퍼파라미터 * log\frac{1-r_{j}}{r_{j}}$

<에이다부스트 예측>

$\hat{y} = argmax\Sigma_{j=1}^{N}\alpha_{j}$

N : 예측기 수

In [16]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),n_estimators=200,algorithm='SAMME.R',learning_rate=0.5)
ada_clf.fit(X_train,y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

### 7.5.2 그레이디언트 부스팅

그레이디언트 부스팅 : 에이다부스트처럼 앙상블에 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가하지만, 반복마다 샘플의 가중치를 수정하는 대신 이전 예측기가 만든 잔여 오차에 새로운 예측기를 학습시킨다.

In [17]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(max_depth=2)

In [19]:
# 두번째 예측기가 만든 잔여 오차에 세 번째 회귀 모델 훈련

y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=2)

In [ ]:
# 모든 트리의 예측의 합
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1,tree_reg2,tree_reg3))

트리가 앙상블에 추가될수록 앙상블의 예측이 점점 좋아진다.

In [21]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2,n_estimators=3,learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

축소 : learning_rate 매개변수를 낮게 설정하여 각 트리의 기여 정도를 조절하면 앙상블을 훈련 세트에 학습시키기 위해 많은 트리가 필요하지만 일반적으로 예측의 성능은 좋아지는 방법

In [22]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X,y)
gbrt = GradientBoostingRegressor(max_depth=2,n_estimators=120)
gbrt.fit(X_train,y_train)

errors = [mean_squared_error(y_val,y_pred) for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators=np.argmin(errors)+1

gbrt_best = GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators)
gbrt_best.fit(X_train,y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=58)

In [23]:
# 조기 종료 구현 (warm_start = True)

gbrt = GradientBoostingRegressor(max_depth=2,warm_start=True)

min_val_error = float('inf')
error_going_up = 0
for n_estimators in range(1,120):
  gbrt.n_estimators = n_estimators
  gbrt.fit(X_train,y_train)
  y_pred = gbrt.predict(X_val)
  val_error = mean_squared_error(y_val,y_pred)
  if val_error < min_val_error:
    min_val_error = val_error
    error_going_up = 0
  else:
    error_going_up += 1
    if error_going_up == 5:
      break #조기 종료

확률적 그레이디언트 부스팅 : 각 트리가 훈련할 때 사용할 훈련 샘플의 비율을 지정한다. 편향이 높아지는 대신 분산이 낮아지고 훈련 속도가 높아진다.

In [24]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train,y_train)
y_pred = xgb_reg.predict(X_val)

[03:52:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [25]:
xgb_reg.fit(X_train,y_train,eval_set=[(X_val,y_val)],early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)

[03:53:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.46686
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:0.43838
[2]	validation_0-rmse:0.41451
[3]	validation_0-rmse:0.394289
[4]	validation_0-rmse:0.376805
[5]	validation_0-rmse:0.361437
[6]	validation_0-rmse:0.348761
[7]	validation_0-rmse:0.338325
[8]	validation_0-rmse:0.329876
[9]	validation_0-rmse:0.322816
[10]	validation_0-rmse:0.316066
[11]	validation_0-rmse:0.310561
[12]	validation_0-rmse:0.305344
[13]	validation_0-rmse:0.301993
[14]	validation_0-rmse:0.298748
[15]	validation_0-rmse:0.29632
[16]	validation_0-rmse:0.292897
[17]	validation_0-rmse:0.29011
[18]	validation_0-rmse:0.287372
[19]	validation_0-rmse:0.284494
[20]	validation_0-rmse:0.282344
[21]	validation_0-rmse:0.280072
[22]	validation_0-rmse:0.277716
[23]	validation_0-rmse:0.276656
[24]	validation_0-rmse:0.274666
[25]	validation_0-rmse:0

## 7.6 스태킹

앙상블에 속한 모든 예측기의 예측을 취합하는 간단한 함수를 사용하는 대신 취합하는 모델을 훈련시키고 싶다는 아이디어에서 출발한다.

새로운 샘플에 회귀 작업을 수행하는 앙상블에서, 기존의 세 예측기는 각각 다른 값을 예측하고 마지막 예측기(블렌더 또는 메타 학습기)가 이 예측을 입력으로 받아 최종 예측을 만든다. 블렌더는 일반적으로 홀드아웃 세트를 사용해서 학습시킨다. 먼저 훈련 세트를 2개의 서브셋으로 나누고, 첫번째 서브셋은 첫번째 레이어의 예측을 훈련시키기 위해 사용한다. 그 다음 첫번째 레이어의 예측기를 사용해 두번째(홀드아웃) 세트에 대한 예측을 만든다. 홀드아웃 세트의 각 샘플에 대해 3개의 예측값이 있는데, 타깃값은 그대로 쓰고 앞에서 예측한 값을 입력 특성으로 사용하는 새로운 훈련 세트를 만들 수 있다.